# Drive mount

In [2]:
from google.colab import drive
import os


drive.mount('/content/drive', force_remount=True)
folder_path = '/content/drive/My Drive/MSc Computer Science/Master Thesis/trial_data/'

for f in os.listdir(folder_path):
    print(f)

Mounted at /content/drive
diachronic.xlsx
speaker1.xlsx
speaker2.xlsx
speaker3.xlsx
speaker4.xlsx


# Preamble

In [1]:
#!pip install numpy==1.26.4 pandas==2.2.2 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.2
    Uninstalling tzdata-2025.2:
      Successfully uninstalled tzdata-2025.2
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2


In [1]:
!pip install tomotopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 89.0 MB/s eta 0:00:00


In [3]:
!python -m spacy download nb_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 30.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('nb_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# Helpers

In [5]:
from scipy.spatial.distance import jensenshannon
import spacy
import numpy as np

nlp = spacy.load("nb_core_news_md")

def compute_jsd_for_lemma(distr_r, distr_f, word_forms):
    def average_distribution(forms, distr):
        vectors = [distr[w] for w in forms if w in distr]
        if not vectors:
            return None

        return np.mean(np.stack(vectors), axis=0)

    lemmatized_forms = set()
    for word in word_forms:
        doc = nlp(word.lower())
        for token in doc:
            if token.is_alpha and not token.is_stop:
                lemmatized_forms.add(token.lemma_)

    p = average_distribution(lemmatized_forms, distr_r)
    q = average_distribution(lemmatized_forms, distr_f)

    return jensenshannon(p, q)

In [6]:
import re

def get_word_forms_by_stem(usages_a, usages_b, stem):
    all_usages = usages_a + usages_b
    word_forms = set()

    for text in all_usages:
        tokens = re.findall(r'\b\w+\b', text.lower())
        for token in tokens:
            if token.startswith(stem):
                word_forms.add(token)

    return sorted(word_forms)

In [7]:
from collections import Counter
import spacy

nlp = spacy.load("nb_core_news_md")

def compute_noveltyy(ref_usages, foc_usages, all_topics, alpha=1.0):
    def assign_sense_to_doc(doc_text, topics):
      doc = nlp(doc_text.lower())
      words = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

      topic_scores = []
      for i, topic in enumerate(topics):
          score = sum(topic.get(word, 0) for word in words)
          topic_scores.append((i, score))

      best_topic_i, best_score = max(topic_scores, key=lambda x: x[1])
      return best_topic_i, best_score, topic_scores

    ref_assignments = [assign_sense_to_doc(doc, all_topics)[0] for doc in ref_usages]
    foc_assignments = [assign_sense_to_doc(doc, all_topics)[0] for doc in foc_usages]

    T = set(range(len(all_topics)))
    T_size = len(T)

    count_ref = Counter(ref_assignments)
    print('count ref', count_ref)
    count_foc = Counter(foc_assignments)
    print('count foc', count_foc)

    # Total counts
    total_ref = sum(count_ref.values())
    total_foc = sum(count_foc.values())

    novelty_scores = {}
    for t in T: # Apply additive smoothing as per equation
        p_f = (count_foc.get(t, 0) + alpha / T_size) / (total_foc + alpha)
        p_r = (count_ref.get(t, 0) + alpha / T_size) / (total_ref + alpha)

        novelty_scores[t] = (p_f - p_r) / p_r if p_r > 0 else float('inf')

    max_novelty_sense = max(novelty_scores, key=novelty_scores.get)
    max_novelty = novelty_scores[max_novelty_sense]

    return max_novelty_sense, max_novelty, novelty_scores


In [24]:
import spacy
import tomotopy as tp
import numpy as np
from collections import defaultdict

nlp = spacy.load("nb_core_news_md")

def lemmatize_and_filter(text):
    doc = nlp(text.lower())
    return [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

def train_model(usages_a, usages_b, iterations=1000, seed=42):
    texts = [lemmatize_and_filter(doc) for doc in usages_a + usages_b]

    model = tp.HDPModel(tw=tp.TermWeight.ONE, seed=seed)
    for text in texts:
        if text:
            model.add_doc(text)

    model.train(iter=iterations, workers=1)
    return model

In [9]:
def get_word_topic_distributions(model, doc_indices):
    word_topic_counts = defaultdict(lambda: np.zeros(model.k))

    for i in doc_indices:
        doc = model.docs[i]
        for word_id, topic_id in zip(doc.words, doc.topics):
            word = model.vocabs[word_id]
            word_topic_counts[word][topic_id] += 1

    word_topic_dists = {}
    for word, topic_counts in word_topic_counts.items():
        total = topic_counts.sum()
        if total > 0:
            word_topic_dists[word] = topic_counts / total

    return word_topic_dists


def get_topic_word_lists(model, num_words=10):
    topic_dicts = []
    for k in range(model.k):
        topic_words = model.get_topic_words(k, top_n=num_words)
        topic_dict = {word: weight for word, weight in topic_words}
        topic_dicts.append(topic_dict)
    return topic_dicts

In [10]:
def word_topic_distributions_corpus_pair(model, usages_a, usages_b):
    index_a = list(range(len(usages_a)))
    index_b = list(range(len(usages_a), len(usages_a) + len(usages_b)))

    dist_a = get_word_topic_distributions(model, index_a)
    dist_b = get_word_topic_distributions(model, index_b)

    return dist_a, dist_b

# Experiment


## Diachronic

In [11]:
import pandas as pd

filename = 'diachronic.xlsx'
folder_path = '/content/drive/My Drive/MSc Computer Science/Master Thesis/trial_data/'

df_d = pd.read_excel(folder_path + filename)
# df_d

In [12]:
tg1_usages = df_d[df_d['usage_id'].str.contains("tg1", na=False)]['text'].tolist()
tg2_usages = df_d[df_d['usage_id'].str.contains("tg2", na=False)]['text'].tolist()

In [25]:
model = train_model(tg1_usages, tg2_usages)

### JSD

In [14]:
tw = "bærekraft"
words_forms_d = get_word_forms_by_stem(tg1_usages, tg2_usages, tw)
words_forms_d

['bærekraft',
 'bærekraftig',
 'bærekraftige',
 'bærekraftmål',
 'bærekraftsmålene',
 'bærekraftutfordringene']

In [27]:
distr_tg1, distr_tg2 = word_topic_distributions_corpus_pair(model, tg1_usages, tg2_usages)

In [30]:
compute_jsd_for_lemma(distr_tg1, distr_tg2, words_forms_d) # np.float64(0.8191556014513339)

0.7821977034161408

### Novelty

In [31]:
dia_topics = get_topic_word_lists(model, num_words=10)

In [32]:
len(dia_topics)

62

In [33]:
max_novelty_sense, max_novelty_score, novelty_scores = compute_noveltyy(
    tg1_usages, tg2_usages, dia_topics)

print("Most novel sense index:", max_novelty_sense)
print("Novelty score of the most novel sense:", max_novelty_score)
print("All novelty scores per sense:", novelty_scores)

count ref Counter({53: 3, 45: 3, 29: 2, 12: 1, 36: 1, 47: 1, 50: 1, 33: 1, 18: 1, 31: 1, 14: 1, 26: 1, 35: 1, 42: 1, 49: 1, 54: 1, 38: 1, 51: 1, 22: 1, 32: 1, 34: 1, 11: 1, 37: 1, 27: 1, 40: 1})
count foc Counter({28: 1, 49: 1, 7: 1, 25: 1, 44: 1, 1: 1, 41: 1, 4: 1, 45: 1, 3: 1, 5: 1, 24: 1, 19: 1, 30: 1, 9: 1, 8: 1, 2: 1, 6: 1, 52: 1, 16: 1, 15: 1, 46: 1, 0: 1, 17: 1, 20: 1, 21: 1, 10: 1, 13: 1, 39: 1, 23: 1})
Most novel sense index: 0
Novelty score of the most novel sense: 62.00000000000001
All novelty scores per sense: {0: 62.00000000000001, 1: 62.00000000000001, 2: 62.00000000000001, 3: 62.00000000000001, 4: 62.00000000000001, 5: 62.00000000000001, 6: 62.00000000000001, 7: 62.00000000000001, 8: 62.00000000000001, 9: 62.00000000000001, 10: 62.00000000000001, 11: -0.9841269841269841, 12: -0.9841269841269841, 13: 62.00000000000001, 14: -0.9841269841269841, 15: 62.00000000000001, 16: 62.00000000000001, 17: 62.00000000000001, 18: -0.9841269841269841, 19: 62.00000000000001, 20: 62.000000

count ref Counter({39: 2, 27: 2, 55: 2, 23: 2, 12: 1, 38: 1, 30: 1, 46: 1, 37: 1, 0: 1, 24: 1, 26: 1, 15: 1, 1: 1, 32: 1, 29: 1, 6: 1, 44: 1, 14: 1, 22: 1, 10: 1, 2: 1, 20: 1, 51: 1, 47: 1, 4: 1})
count foc Counter({55: 3, 43: 1, 41: 1, 34: 1, 19: 1, 13: 1, 42: 1, 11: 1, 36: 1, 25: 1, 21: 1, 49: 1, 33: 1, 35: 1, 18: 1, 17: 1, 9: 1, 5: 1, 28: 1, 48: 1, 40: 1, 31: 1, 8: 1, 16: 1, 45: 1, 3: 1, 50: 1, 7: 1})
Most novel sense index: 3
Novelty score of the most novel sense: 63.0
All novelty scores per sense: {0: -0.984375, 1: -0.984375, 2: -0.984375, 3: 63.0, 4: -0.984375, 5: 63.0, 6: -0.984375, 7: 63.0, 8: 63.0, 9: 63.0, 10: -0.984375, 11: 63.0, 12: -0.984375, 13: 63.0, 14: -0.984375, 15: -0.984375, 16: 63.0, 17: 63.0, 18: 63.0, 19: 63.0, 20: -0.984375, 21: 63.0, 22: -0.984375, 23: -0.9921259842519685, 24: -0.984375, 25: 63.0, 26: -0.984375, 27: -0.9921259842519685, 28: 63.0, 29: -0.984375, 30: -0.984375, 31: 63.0, 32: -0.984375, 33: 63.0, 34: 63.0, 35: 63.0, 36: 63.0, 37: -0.984375, 38: -0.984375, 39: -0.9921259842519685, 40: 63.0, 41: 63.0, 42: 63.0, 43: 63.0, 44: -0.984375, 45: 63.0, 46: -0.984375, 47: -0.984375, 48: 63.0, 49: 63.0, 50: 63.0, 51: -0.984375, 52: 0.0, 53: 0.0, 54: 0.0, 55: 0.49606299212598426, 56: 0.0, 57: 0.0, 58: 0.0, 59: 0.0, 60: 0.0, 61: 0.0, 62: 0.0}

## Speaker 1

In [34]:
import pandas as pd

filename = 'speaker1.xlsx'
folder_path = '/content/drive/My Drive/MSc Computer Science/Master Thesis/trial_data/'

df_s1 = pd.read_excel(folder_path + filename)
#df_s1

In [35]:
gs1_usages = df_s1[df_s1['usage_id'].str.contains("general", na=False)]['text'].tolist()
s1_usages = df_s1[df_s1['usage_id'].str.contains("speaker", na=False)]['text'].tolist()

In [40]:
model_s1 = train_model(gs1_usages, s1_usages)

### JSD

In [37]:
tw = "bærekraft"
word_forms_s1 = get_word_forms_by_stem(gs1_usages, s1_usages, tw)
word_forms_s1

# ['bærekraft','bærekraftbegrepene','bærekraften','bærekraftig','bærekraftige','bærekraftindikator','bærekraftindikatorer','bærekraftperspektiv','bærekraftsindikatorer','bærekraftskriteriene','bærekraftskvalitet','bærekraftsmålet']

['bærekraft',
 'bærekraftbegrepene',
 'bærekraften',
 'bærekraftig',
 'bærekraftige',
 'bærekraftindikator',
 'bærekraftindikatorer',
 'bærekraftperspektiv',
 'bærekraftsindikatorer',
 'bærekraftskriteriene',
 'bærekraftskvalitet',
 'bærekraftsmålet']

In [41]:
distr_gs1, distr_s1 = word_topic_distributions_corpus_pair(model_s1, gs1_usages, s1_usages)

In [42]:
compute_jsd_for_lemma(distr_gs1, distr_s1, word_forms_s1) # 0.7631599710096147

0.7505834648136037

### Novelty

In [43]:
s1_topics = get_topic_word_lists(model_s1)

In [44]:
len(s1_topics)

59

In [45]:
max_novelty_sense, max_novelty_score, novelty_scores = compute_noveltyy(
    gs1_usages, s1_usages, s1_topics)

print("Most novel sense index:", max_novelty_sense)
print("Novelty score of the most novel sense:", max_novelty_score) # Novelty score of the most novel sense: 61.0
print("All novelty scores per sense:", novelty_scores)

count ref Counter({39: 3, 24: 2, 30: 1, 7: 1, 35: 1, 2: 1, 37: 1, 29: 1, 38: 1, 13: 1, 46: 1, 21: 1, 20: 1, 42: 1, 16: 1, 54: 1, 15: 1, 25: 1, 10: 1, 17: 1, 23: 1, 1: 1, 4: 1, 8: 1, 45: 1, 18: 1, 14: 1})
count foc Counter({45: 3, 50: 2, 39: 2, 40: 1, 6: 1, 11: 1, 22: 1, 27: 1, 31: 1, 3: 1, 44: 1, 9: 1, 34: 1, 28: 1, 12: 1, 5: 1, 0: 1, 41: 1, 47: 1, 26: 1, 48: 1, 32: 1, 55: 1, 33: 1, 19: 1, 36: 1})
Most novel sense index: 50
Novelty score of the most novel sense: 118.0
All novelty scores per sense: {0: 59.0, 1: -0.9833333333333333, 2: -0.9833333333333333, 3: 59.0, 4: -0.9833333333333333, 5: 59.0, 6: 59.0, 7: -0.9833333333333333, 8: -0.9833333333333333, 9: 59.0, 10: -0.9833333333333333, 11: 59.0, 12: 59.0, 13: -0.9833333333333333, 14: -0.9833333333333333, 15: -0.9833333333333333, 16: -0.9833333333333333, 17: -0.9833333333333333, 18: -0.9833333333333333, 19: 59.0, 20: -0.9833333333333333, 21: -0.9833333333333333, 22: 59.0, 23: -0.9833333333333333, 24: -0.9915966386554621, 25: -0.983333333

count ref Counter({29: 2, 18: 2, 31: 1, 4: 1, 32: 1, 0: 1, 25: 1, 51: 1, 30: 1, 44: 1, 23: 1, 21: 1, 49: 1, 24: 1, 43: 1, 37: 1, 50: 1, 11: 1, 27: 1, 14: 1, 28: 1, 33: 1, 1: 1, 12: 1, 6: 1, 13: 1, 38: 1, 20: 1})
count foc Counter({13: 2, 18: 2, 45: 2, 41: 1, 19: 1, 52: 1, 46: 1, 48: 1, 3: 1, 5: 1, 22: 1, 26: 1, 39: 1, 8: 1, 35: 1, 34: 1, 7: 1, 10: 1, 47: 1, 42: 1, 9: 1, 40: 1, 36: 1, 2: 1, 15: 1, 16: 1, 17: 1})
Most novel sense index: 45
Novelty score of the most novel sense: 120.0
All novelty scores per sense: {0: -0.9836065573770493, 1: -0.9836065573770493, 2: 60.0, 3: 60.0, 4: -0.9836065573770493, 5: 60.0, 6: -0.9836065573770493, 7: 60.0, 8: 60.0, 9: 60.0, 10: 60.0, 11: -0.9836065573770493, 12: -0.9836065573770493, 13: 0.9836065573770493, 14: -0.9836065573770493, 15: 60.0, 16: 60.0, 17: 60.0, 18: 0.0, 19: 60.0, 20: -0.9836065573770493, 21: -0.9836065573770493, 22: 60.0, 23: -0.9836065573770493, 24: -0.9836065573770493, 25: -0.9836065573770493, 26: 60.0, 27: -0.9836065573770493, 28: -0.9836065573770493, 29: -0.9917355371900827, 30: -0.9836065573770493, 31: -0.9836065573770493, 32: -0.9836065573770493, 33: -0.9836065573770493, 34: 60.0, 35: 60.0, 36: 60.0, 37: -0.9836065573770493, 38: -0.9836065573770493, 39: 60.0, 40: 60.0, 41: 60.0, 42: 60.0, 43: -0.9836065573770493, 44: -0.9836065573770493, 45: 120.0, 46: 60.0, 47: 60.0, 48: 60.0, 49: -0.9836065573770493, 50: -0.9836065573770493, 51: -0.9836065573770493, 52: 60.0, 53: 0.0, 54: 0.0, 55: 0.0, 56: 0.0, 57: 0.0, 58: 0.0, 59: 0.0}

## Speaker 2

In [46]:
filename = 'speaker2.xlsx'
folder_path = '/content/drive/My Drive/MSc Computer Science/Master Thesis/trial_data/'

df_s2 = pd.read_excel(folder_path + filename)
#df_s2

In [47]:
gs2_usages = df_s2[df_s2['usage_id'].str.contains("general", na=False)]['text'].tolist()
s2_usages = df_s2[df_s2['usage_id'].str.contains("speaker", na=False)]['text'].tolist()

In [48]:
model_s2 = train_model(gs2_usages, s2_usages, seed=1905)

### JSD

In [49]:
word_forms_s2 = get_word_forms_by_stem(gs2_usages, s2_usages, tw)
word_forms_s2

['bærekraft', 'bærekraftig', 'bærekraftige']

In [50]:
distr_gs2, distr_s2 = word_topic_distributions_corpus_pair(model_s2, gs2_usages, s2_usages)

In [51]:
compute_jsd_for_lemma(distr_gs2, distr_s2, word_forms_s2) # 0.7864538886097446

0.7819907604003781

### Novelty

In [52]:
s2_topics = get_topic_word_lists(model_s1)

In [53]:
len(s2_topics)

59

In [54]:
max_novelty_sense, max_novelty_score, novelty_scores = compute_noveltyy(
    gs2_usages, s2_usages, s2_topics)

print("Most novel sense index:", max_novelty_sense)
print("Novelty score of the most novel sense:", max_novelty_score)

count ref Counter({39: 25, 27: 1, 14: 1, 2: 1, 45: 1, 35: 1})
count foc Counter({39: 23, 45: 4, 46: 1, 25: 1, 13: 1})
Most novel sense index: 13
Novelty score of the most novel sense: 59.0


count ref Counter({18: 22, 20: 2, 32: 2, 3: 1, 0: 1, 10: 1, 13: 1})
count foc Counter({18: 23, 13: 4, 44: 1, 27: 1, 30: 1})
Most novel sense index: 27
Novelty score of the most novel sense: 60.0

## Speaker 3

In [55]:
filename = 'speaker3.xlsx'
folder_path = '/content/drive/My Drive/MSc Computer Science/Master Thesis/trial_data/'

df_s3 = pd.read_excel(folder_path + filename)
#df_s3

In [56]:
gs3_usages = df_s3[df_s3['usage_id'].str.contains("general", na=False)]['text'].tolist()
s3_usages = df_s3[df_s3['usage_id'].str.contains("speaker", na=False)]['text'].tolist()

In [58]:
model_s3 = train_model(gs3_usages, s3_usages)

### JSD

In [63]:
tw = 'bærekraft'
word_forms_s3 = get_word_forms_by_stem(gs3_usages, s3_usages, tw)
word_forms_s3

# ['bærekraft', 'bærekraften', 'bærekraftig', 'bærekraftige', 'bærekraftmål', 'bærekraftmålene', 'bærekraftsmål', 'bærekraftsmålene']

['bærekraft',
 'bærekraften',
 'bærekraftig',
 'bærekraftige',
 'bærekraftmål',
 'bærekraftmålene',
 'bærekraftsmål',
 'bærekraftsmålene']

In [64]:
distr_gs3, distr_s3 = word_topic_distributions_corpus_pair(model_s3, gs3_usages, s3_usages)

In [65]:
compute_jsd_for_lemma(distr_gs3, distr_s3, word_forms_s3) # 0.8268549337358913

0.8211156936308213

### Novelty

In [66]:
s3_topics = get_topic_word_lists(model_s3)

In [67]:
len(s3_topics)

62

In [68]:
max_novelty_sense, max_novelty_score, novelty_scores = compute_noveltyy(
    gs3_usages, s3_usages, s3_topics)

print("Most novel sense index:", max_novelty_sense)
print("Novelty score of the most novel sense:", max_novelty_score)

count ref Counter({41: 2, 26: 2, 39: 1, 43: 1, 28: 1, 21: 1, 53: 1, 23: 1, 9: 1, 17: 1, 7: 1, 37: 1, 6: 1, 49: 1, 1: 1, 0: 1, 25: 1, 38: 1, 47: 1, 27: 1, 3: 1, 32: 1, 30: 1, 10: 1, 24: 1, 35: 1, 16: 1, 14: 1})
count foc Counter({51: 2, 55: 2, 20: 1, 56: 1, 12: 1, 40: 1, 44: 1, 4: 1, 29: 1, 8: 1, 52: 1, 15: 1, 42: 1, 39: 1, 36: 1, 31: 1, 11: 1, 18: 1, 45: 1, 46: 1, 5: 1, 0: 1, 13: 1, 48: 1, 34: 1, 33: 1, 2: 1, 22: 1})
Most novel sense index: 51
Novelty score of the most novel sense: 124.00000000000001


count ref Counter({42: 16, 28: 3, 8: 2, 6: 2, 32: 1, 7: 1, 35: 1, 2: 1, 11: 1, 40: 1, 23: 1})
count foc Counter({42: 19, 13: 2, 9: 2, 28: 2, 4: 1, 14: 1, 6: 1, 45: 1, 2: 1})
Most novel sense index: 9
Novelty score of the most novel sense: 102.0
Average novelty score: 6.852667635153881

## Speaker 4

In [69]:
filename = 'speaker4.xlsx'
folder_path = '/content/drive/My Drive/MSc Computer Science/Master Thesis/trial_data/'

df_s4 = pd.read_excel(folder_path + filename)
#df_s4

In [70]:
gs4_usages = df_s4[df_s4['usage_id'].str.contains("general", na=False)]['text'].tolist()
s4_usages = df_s4[df_s4['usage_id'].str.contains("speaker", na=False)]['text'].tolist()

In [77]:
model_s4 = train_model(gs4_usages, s4_usages)

### JSD

In [72]:
word_forms_s4 = get_word_forms_by_stem(gs4_usages, s4_usages, tw)
word_forms_s4
# ['bærekraft','bærekraften','bærekraftig', 'bærekraftige', 'bærekraftighetsspørsmålene', 'bærekraftsinformasjon', 'bærekraftskrav', 'bærekraftsmål', 'bærekraftsmålene']

['bærekraft',
 'bærekraften',
 'bærekraftig',
 'bærekraftige',
 'bærekraftighetsspørsmålene',
 'bærekraftsinformasjon',
 'bærekraftskrav',
 'bærekraftsmål',
 'bærekraftsmålene']

In [79]:
distr_gs4, distr_s4 = word_topic_distributions_corpus_pair(model_s4, gs4_usages, s4_usages)

In [80]:
compute_jsd_for_lemma(distr_gs4, distr_s4, word_forms_s4) # 0.8152904133872028

0.8036280476082095

### Novelty

In [81]:
s4_topics = get_topic_word_lists(model_s4)

In [82]:
len(s4_topics)

62

In [83]:
max_novelty_sense, max_novelty_score, novelty_scores = compute_noveltyy(
    gs4_usages, s4_usages, s4_topics)

print("Most novel sense index:", max_novelty_sense)
print("Novelty score of the most novel sense:", max_novelty_score)

count ref Counter({3: 2, 50: 1, 14: 1, 9: 1, 29: 1, 13: 1, 5: 1, 37: 1, 30: 1, 49: 1, 6: 1, 31: 1, 41: 1, 32: 1, 40: 1, 2: 1, 44: 1, 27: 1, 47: 1, 4: 1, 26: 1, 28: 1, 20: 1, 18: 1, 1: 1, 21: 1, 16: 1, 7: 1, 55: 1})
count foc Counter({20: 2, 22: 2, 53: 2, 35: 1, 17: 1, 25: 1, 8: 1, 45: 1, 54: 1, 43: 1, 12: 1, 39: 1, 51: 1, 52: 1, 10: 1, 42: 1, 48: 1, 33: 1, 46: 1, 19: 1, 24: 1, 38: 1, 11: 1, 34: 1, 0: 1, 15: 1, 23: 1})
Most novel sense index: 22
Novelty score of the most novel sense: 124.00000000000001


count ref Counter({47: 2, 43: 1, 20: 1, 5: 1, 12: 1, 6: 1, 1: 1, 19: 1, 55: 1, 23: 1, 49: 1, 36: 1, 28: 1, 59: 1, 14: 1, 22: 1, 39: 1, 53: 1, 8: 1, 21: 1, 11: 1, 48: 1, 17: 1, 16: 1, 45: 1, 0: 1, 15: 1, 9: 1, 32: 1})
count foc Counter({17: 2, 41: 2, 10: 1, 51: 1, 38: 1, 35: 1, 31: 1, 56: 1, 42: 1, 34: 1, 52: 1, 46: 1, 3: 1, 18: 1, 37: 1, 44: 1, 24: 1, 54: 1, 33: 1, 30: 1, 45: 1, 7: 1, 25: 1, 2: 1, 13: 1, 27: 1, 40: 1, 50: 1})
Most novel sense index: 41
Novelty score of the most novel sense: 128.0